In [24]:
import pickle
import numpy as np
import networkx as nx
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

In [11]:
# Fonction pour charger un fichier .pkl
def load_pkl_file(file_path):
    with open(file_path, 'rb') as f:
        data = pickle.load(f)
    return data

# Exemple de chemin de fichier .pkl
file_path = "dataset_trainA.pkl"  # Remplacez par le chemin de votre fichier .pkl

data = load_pkl_file(file_path)


In [9]:
data.iloc[0]["flux_dégradé"]

,Abbesses,Aimé Césaire,Alexandre Dumas,Alma - Marceau,Alésia,Anatole France,Anvers,Argentine,Arts et Métiers,Assemblée nationale,...,Villejuif - Paul Vaillant-Couturier,Villiers,Volontaires,Voltaire,Wagram,École Militaire,École vétérinaire de Maisons-Alfort,Église d'Auteuil,Église de Pantin,Étienne Marcel
Abbesses,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Aimé Césaire,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Alexandre Dumas,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Alma - Marceau,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Alésia,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
École Militaire,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
École vétérinaire de Maisons-Alfort,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Église d'Auteuil,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Église de Pantin,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
print(data.iloc[0]["flux_dégradé"])  # Affiche la première paire de matrices

                                     Abbesses  Aimé Césaire  Alexandre Dumas  \
Abbesses                                  0.0           0.0              0.0   
Aimé Césaire                              0.0           0.0              0.0   
Alexandre Dumas                           0.0           0.0              0.0   
Alma - Marceau                            0.0           0.0              0.0   
Alésia                                    0.0           0.0              0.0   
...                                       ...           ...              ...   
École Militaire                           0.0           0.0              0.0   
École vétérinaire de Maisons-Alfort       0.0           0.0              0.0   
Église d'Auteuil                          0.0           0.0              0.0   
Église de Pantin                          0.0           0.0              0.0   
Étienne Marcel                            0.0           0.0              0.0   

                                     Al

In [13]:
adj_matrices = []
flux_matrices = []

for i, (G, flux) in enumerate(zip(data["plan_metro_degradé"], data["flux_dégradé"])):
    
    # Convertir le graphe en une matrice d'adjacence
    adj_matrix = nx.adjacency_matrix(G).todense()  # Convertir en format dense
    
    # Convertir la matrice d'adjacence en numpy
    adj_matrix_np = np.array(adj_matrix)
    
    # Convertir la matrice de flux en numpy
    flux_matrix_np = np.array(flux)
    
    # Convertir les matrices en tensor PyTorch
    adj_tensor = torch.tensor(adj_matrix_np, dtype=torch.float32)
    flux_tensor = torch.tensor(flux_matrix_np, dtype=torch.float32)
    
    # Ajouter les matrices dans les listes
    adj_matrices.append(adj_tensor)  # Liste de matrices d'adjacence
    flux_matrices.append(flux_tensor)  # Liste de matrices de flux
    
    # Optionnel : Afficher les matrices d'adjacence et de flux pour chaque plan
    print(f"Plan de métro {i + 1} :")
    print(f"Adjacency Matrix Tensor :\n", adj_tensor)
    print(adj_tensor.shape)
    print(f"Flux Matrix Tensor :\n", flux_tensor)
    print(flux_tensor.shape)


Plan de métro 1 :
Adjacency Matrix Tensor :
 tensor([[0., 1., 1.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
torch.Size([307, 307])
Flux Matrix Tensor :
 tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
torch.Size([307, 307])
Plan de métro 2 :
Adjacency Matrix Tensor :
 tensor([[0., 1., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
torch.Size([307, 307])
Flux Matrix Tensor :
 tensor([[0., 0., 0.,  ..., 0., 0., 

In [38]:
# Fonction pour normaliser la matrice d'adjacence
def normalize_adj(adj):
    adj = adj + torch.eye(adj.size(0))  # Ajout des boucles propres (self-loops)
    degree = torch.sum(adj, dim=1)
    d_inv_sqrt = torch.pow(degree, -0.5).flatten()
    d_inv_sqrt[torch.isinf(d_inv_sqrt)] = 0.0
    d_mat_inv_sqrt = torch.diag(d_inv_sqrt)
    return torch.mm(torch.mm(d_mat_inv_sqrt, adj), d_mat_inv_sqrt)

def normalize_features(features):
    mean = torch.mean(features, dim=0)
    std = torch.std(features, dim=0)
    std[std == 0] = 1  # Évite la division par 0
    normalized_features = (features - mean) / std
    return normalized_features, mean, std

def denormalize_output(output, mean, std):
    return (output * std) + mean

In [32]:
class CustomLoss(nn.Module):
    def __init__(self):
        super(CustomLoss, self).__init__()
    
    def forward(self, predictions, targets):
        # Masquer les valeurs où target == 0
        mask = (targets != 0).float()  # Masque binaire : 1 si target != 0, sinon 0
        
        # Différence uniquement là où target != 0
        diff = predictions - targets
        
        # Appliquer le masque
        masked_diff = diff * mask
        
        # Calculer la somme des différences au carré (MSE) seulement pour les éléments valides
        loss = torch.sum(masked_diff ** 2) / torch.sum(mask)  # Moyenne normalisée
        return loss


In [33]:
# Définition de la couche GCN
class GCNLayer(nn.Module):
    def __init__(self, in_features, out_features):
        super(GCNLayer, self).__init__()
        self.linear = nn.Linear(in_features, out_features)

    def forward(self, x, adj):
        # Application de la normalisation et propagation
        out = torch.matmul(adj, x)  # Multiplication par la matrice de connectivité
        out = self.linear(out)  # Transformation linéaire
        return F.relu(out)

# Définition du modèle GCN complet
class GCN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCN, self).__init__()
        self.gcn1 = GCNLayer(input_dim, hidden_dim)  # Première couche
        self.gcn2 = GCNLayer(hidden_dim, output_dim)  # Deuxième couche

    def forward(self, x, adj):
        h = self.gcn1(x, adj)  # Propagation dans la première couche
        h = self.gcn2(h, adj)  # Propagation dans la deuxième couche
        return h  # Sortie du modè


In [34]:
class GATLayer(nn.Module):
    def __init__(self, in_features, out_features):
        super(GATLayer, self).__init__()
        self.W = nn.Linear(in_features, out_features, bias=False)
        self.a = nn.Linear(2 * out_features, 1, bias=False)

    def forward(self, x, adj):
        h = self.W(x)
        N = h.size(0)
        a_input = torch.cat([h.repeat(1, N).view(N * N, -1), h.repeat(N, 1)], dim=1)
        e = F.leaky_relu(self.a(a_input).view(N, N))
        attention = F.softmax(e.masked_fill(adj == 0, -1e9), dim=1)
        h_prime = torch.matmul(attention, h)
        return F.relu(h_prime)

class GAT(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GAT, self).__init__()
        self.gat1 = GATLayer(input_dim, hidden_dim)
        self.gat2 = GATLayer(hidden_dim, output_dim)

    def forward(self, x, adj):
        h = self.gat1(x, adj)
        h = self.gat2(h, adj)
        return h


In [49]:
# Hyperparamètres
input_dim = 307  # Exemple : nombre d'attributs par station
hidden_dim = 64  # Dimension cachée
output_dim = 307  # Flux de passagers entre chaque paire de stations
learning_rate = 0.01
nb_instances = 1

# Création du modèle GCN
model_gcn = GCN(input_dim, hidden_dim, output_dim)
model_gat = GAT(input_dim, hidden_dim, output_dim)

# Fonction de perte et optimiseur
criterion = CustomLoss()  # Utilise votre fonction de perte personnalisée
optimizer1 = optim.Adam(model_gcn.parameters(), lr=learning_rate)

# Entraînement
epochs = 200

features_matrix = []  # Matrices de caractéristiques (flux dégradé)
adj_list = []         # Matrices d'adjacence (plan métro dégradé)

for i, (graph, flux) in enumerate(zip(data["plan_metro_degradé"], data["flux_dégradé"])):
    # Convertir le graphe en une matrice d'adjacence
    adj_matrix = nx.adjacency_matrix(graph).todense()  # Conversion en dense
    adj_tensor = torch.tensor(np.array(adj_matrix), dtype=torch.float32)  # En Tensor
    
    # Convertir la matrice de flux en Tensor
    flux_tensor = torch.tensor(np.array(flux), dtype=torch.float32)
    
    # Ajouter à la liste
    adj_list.append(adj_tensor)
    features_matrix.append(flux_tensor)

# Normalisation des données d'entrée
normalized_features_matrix = []
mean_std_list = []  # Pour stocker les statistiques (mean et std) pour chaque instance

for features in features_matrix:
    normalized_features, mean, std = normalize_features(features)
    normalized_features_matrix.append(normalized_features)
    mean_std_list.append((mean, std))  # Stocker mean et std pour dénormaliser plus tard

# Normaliser les matrices d'adjacence
normalized_adj_list = [normalize_adj(a) for a in adj_list]

# Exemple de cibles : matrices de flux réelles à prédire
y = [torch.rand((input_dim, input_dim)) for _ in range(nb_instances)]

In [46]:
len(normalized_features_matrix[0])

307

In [51]:
# Boucle d'entraînement
for epoch in range(epochs):
    model_gcn.train()  # Mode entraînement
    epoch_loss = 0  # Suivre la perte totale pour l'époque

    for i in range(len(features_matrix)):  # Utilisation de features_matrix
        features = features_matrix[i]  # Flux pour l'instance i
        adj = adj_list[i]  # Matrice d'adjacence pour l'instance i
        
        # Assurez-vous que les entrées sont des Tensors PyTorch
        features = torch.tensor(features, dtype=torch.float32)
        adj = torch.tensor(adj, dtype=torch.float32)
        
        # Propagation avant
        output = model_gcn(features, adj)  # Prédiction des flux de passagers pour l'instance i
        
        # Calcul de la perte
        loss = criterion(output, y[i])  # Utilisation des vraies valeurs de flux
        epoch_loss += loss.item()
        
        # Rétropropagation et optimisation
        optimizer1.zero_grad()  # Mise à zéro des gradients
        loss.backward()  # Calcul des gradients
        optimizer1.step()  # Mise à jour des paramètres du modèle

    if epoch % 20 == 0:
        print(f'Epoch {epoch}, Loss: {epoch_loss / len(features_matrix)}')

# Récupérer et dénormaliser les sorties après l'entraînement
model_gcn.eval()  # Mode évaluation
final_outputs = []

for i in range(len(normalized_features_matrix)):
    features = normalized_features_matrix[i]
    adj = normalized_adj_list[i]
    
    features = torch.tensor(features, dtype=torch.float32)
    adj = torch.tensor(adj, dtype=torch.float32)
    
    # Propagation avant pour obtenir la sortie
    output = model_gcn(features, adj)
    
    # Dénormalisation de la sortie
    mean, std = mean_std_list[i]
    denormalized_output = denormalize_output(output, mean, std)
    final_outputs.append(denormalized_output)

# Exemple : Impression d'une sortie dénormalisée
print(final_outputs[0])

/tmp/ipykernel_92497/140405885.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  features = torch.tensor(features, dtype=torch.float32)
/tmp/ipykernel_92497/140405885.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  adj = torch.tensor(adj, dtype=torch.float32)


IndexError: list index out of range